**Learn the Vocabulary**

We use the Multi30k dataset for english - german translation. This notebook uses the byte-pair encoding to fit a joint vocabulary for both the English and German language based on the most frequent byte pairs.

In [1]:
from torchtext.datasets import multi30k, Multi30k

multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"
multi30k.URL["test"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/mmt16_task1_test.tar.gz"

data = Multi30k()

/opt/miniforge3/envs/nlp-experiments/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data, val_data, test_data = data

In [3]:
train_data = list(train_data)

In [4]:
train_data[0]

('Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'Two young, White males are outside near many bushes.')

In [5]:
def get_joint_text_corpus(data):
    joint_text = []
    for pair in data:
        joint_text.append(pair[0])
        joint_text.append(pair[1])
    return joint_text

text_joint_en_de = get_joint_text_corpus(train_data)
text_joint_en_de[:10]

['Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'Two young, White males are outside near many bushes.',
 'Mehrere Männer mit Schutzhelmen bedienen ein Antriebsradsystem.',
 'Several men in hard hats are operating a giant pulley system.',
 'Ein kleines Mädchen klettert in ein Spielhaus aus Holz.',
 'A little girl climbing into a wooden playhouse.',
 'Ein Mann in einem blauen Hemd steht auf einer Leiter und putzt ein Fenster.',
 'A man in a blue shirt is standing on a ladder cleaning a window.',
 'Zwei Männer stehen am Herd und bereiten Essen zu.',
 'Two men are at the stove preparing food.']

**How do we turn a text corpus into a fixed length vector?**

With the Byte-Pair Encoding. Let's implement it.

In [6]:
def extract_corpus_from_data(data, language, n_sentences):
    corpus = []
    for i in range(n_sentences):
        corpus.append(data[i][0 if language == "german" else 1])
    return corpus

N_SENTENCES = 1000
english_corpus = extract_corpus_from_data(train_data, "english", N_SENTENCES)
german_corpus = extract_corpus_from_data(train_data, "german", N_SENTENCES)

In [7]:
from byte_pair_encoder import BytePairEncoder
import timeit
from pathlib import Path

data_path = "data/universal_bpe_encoder.pkl"
if not Path(data_path).exists():
    encoder_universal = BytePairEncoder("universal", max_vocab_size=1000, use_start_token=True, use_end_token=True, use_padding_token=True, max_token_len=50)
    time = timeit.timeit(lambda: encoder_universal.learn_vocabulary_from_corpus(text_joint_en_de, n_processes=8), number=1)
    print(f"Time to learn vocabulary for universal BPE: {time:.2f} seconds")
    encoder_universal.save_vocabulary(data_path)
else:
    encoder_universal = BytePairEncoder.load_vocabulary(data_path)

  0%|          | 0/894 [00:00<?, ?it/s]

  2%|▏         | 14/894 [00:27<32:02,  2.18s/it]

KeyboardInterrupt: 